In [7]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from citipy import citipy
# Import the requests library.
import requests
# Import the API key.
from config import weather_api_key
# Import the datetime module from the datetime library.
from datetime import datetime
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
print(url)

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=97824ff516658256d2b4f5c605b47aa0


In [8]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)
lat_lngs

In [9]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [10]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

636

In [11]:
cities

['bethel',
 'illoqqortoormiut',
 'altay',
 'zaraza',
 'rikitea',
 'ilulissat',
 'sabang',
 'ewa beach',
 'husavik',
 'tahta',
 'pitsunda',
 'busselton',
 'mugur-aksy',
 'talnakh',
 'dillon',
 'tete',
 'jamestown',
 'qaanaaq',
 'castro',
 'bredasdorp',
 'fernley',
 'bambanglipuro',
 'ancud',
 'puerto colombia',
 'murgab',
 'yellowknife',
 'cabo san lucas',
 'anadyr',
 'port alfred',
 'tiznit',
 'cherskiy',
 'turbat',
 'mahebourg',
 'namatanai',
 'tabou',
 'padang',
 'albany',
 'salihorsk',
 'shingu',
 'marienburg',
 'atuona',
 'klyuchi',
 'bengkulu',
 'hilo',
 'nikolskoye',
 'amapa',
 'saint-philippe',
 'kapaa',
 'nikolayevsk-na-amure',
 'thompson',
 'pevek',
 'valparaiso',
 'bandar-e lengeh',
 'belushya guba',
 'ushuaia',
 'nhulunbuy',
 'vaini',
 'new norfolk',
 'barentsburg',
 'aanekoski',
 'terrace bay',
 'tiksi',
 'port augusta',
 'yanan',
 'hithadhoo',
 'san jose',
 'georgetown',
 'atambua',
 'lavrentiya',
 'oksfjord',
 'pokhara',
 'punta alta',
 'bushehr',
 'torbay',
 'cape town',

In [12]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1
# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

    # If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | bethel
Processing Record 2 of Set 1 | illoqqortoormiut
City not found. Skipping...
Processing Record 3 of Set 1 | altay
Processing Record 4 of Set 1 | zaraza
Processing Record 5 of Set 1 | rikitea
Processing Record 6 of Set 1 | ilulissat
Processing Record 7 of Set 1 | sabang
Processing Record 8 of Set 1 | ewa beach
Processing Record 9 of Set 1 | husavik
Processing Record 10 of Set 1 | tahta
Processing Record 11 of Set 1 | pitsunda
Processing Record 12 of Set 1 | busselton
Processing Record 13 of Set 1 | mugur-aksy
Processing Record 14 of Set 1 | talnakh
Processing Record 15 of Set 1 | dillon
Processing Record 16 of Set 1 | tete
Processing Record 17 of Set 1 | jamestown
Processing Record 18 of Set 1 | qaanaaq
Processing Record 19 of Set 1 | castro
Processing Record 20 of Set 1 | bredasdorp
Processing Record 21 of Set 1 | fernley
Processing Record 22 of Set 1 | bambanglipuro
Processing Record 23 of

Processing Record 41 of Set 4 | port blair
Processing Record 42 of Set 4 | seoul
Processing Record 43 of Set 4 | kodinsk
Processing Record 44 of Set 4 | aklavik
Processing Record 45 of Set 4 | nome
Processing Record 46 of Set 4 | acari
Processing Record 47 of Set 4 | pangai
Processing Record 48 of Set 4 | mangit
Processing Record 49 of Set 4 | barrow
Processing Record 50 of Set 4 | pangnirtung
Processing Record 1 of Set 5 | faya
Processing Record 2 of Set 5 | caravelas
Processing Record 3 of Set 5 | namibe
Processing Record 4 of Set 5 | caloundra
Processing Record 5 of Set 5 | jalu
Processing Record 6 of Set 5 | mount pleasant
Processing Record 7 of Set 5 | los llanos de aridane
Processing Record 8 of Set 5 | hongjiang
Processing Record 9 of Set 5 | tarakan
Processing Record 10 of Set 5 | kavieng
Processing Record 11 of Set 5 | xingyi
Processing Record 12 of Set 5 | kuche
City not found. Skipping...
Processing Record 13 of Set 5 | soyaux
Processing Record 14 of Set 5 | malwan
City not 

Processing Record 28 of Set 8 | san carlos de bariloche
Processing Record 29 of Set 8 | umm lajj
Processing Record 30 of Set 8 | broome
Processing Record 31 of Set 8 | mayumba
Processing Record 32 of Set 8 | goderich
Processing Record 33 of Set 8 | zhangjiakou
Processing Record 34 of Set 8 | port hedland
Processing Record 35 of Set 8 | nanortalik
Processing Record 36 of Set 8 | jesenice
Processing Record 37 of Set 8 | iqaluit
Processing Record 38 of Set 8 | ljubljana
Processing Record 39 of Set 8 | charters towers
Processing Record 40 of Set 8 | chipiona
Processing Record 41 of Set 8 | ixtapa
Processing Record 42 of Set 8 | pistoia
Processing Record 43 of Set 8 | kenai
Processing Record 44 of Set 8 | saldanha
Processing Record 45 of Set 8 | saint-augustin
Processing Record 46 of Set 8 | galle
Processing Record 47 of Set 8 | takoradi
Processing Record 48 of Set 8 | flinders
Processing Record 49 of Set 8 | mandalgovi
Processing Record 50 of Set 8 | andenes
Processing Record 1 of Set 9 | 

Processing Record 19 of Set 12 | guozhen
Processing Record 20 of Set 12 | kentau
Processing Record 21 of Set 12 | tecpan
Processing Record 22 of Set 12 | moshenskoye
Processing Record 23 of Set 12 | ponta delgada
Processing Record 24 of Set 12 | babanusah
City not found. Skipping...
Processing Record 25 of Set 12 | nampula
Processing Record 26 of Set 12 | bontang
Processing Record 27 of Set 12 | poum
Processing Record 28 of Set 12 | yantal
Processing Record 29 of Set 12 | srandakan
Processing Record 30 of Set 12 | harper
Processing Record 31 of Set 12 | de aar
Processing Record 32 of Set 12 | hailey
Processing Record 33 of Set 12 | chissamba
Processing Record 34 of Set 12 | farah
Processing Record 35 of Set 12 | manicore
Processing Record 36 of Set 12 | khasan
Processing Record 37 of Set 12 | poopo
Processing Record 38 of Set 12 | riohacha
Processing Record 39 of Set 12 | ganzhou
Processing Record 40 of Set 12 | alotau
City not found. Skipping...
Processing Record 41 of Set 12 | saint-

In [13]:
len(city_data)

589

In [14]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Bethel,41.3712,-73.4140,66.56,63,20,5.75,US,2021-10-01 21:30:17
1,Altay,47.8667,88.1167,31.89,69,100,5.14,CN,2021-10-01 21:24:39
2,Zaraza,9.3503,-65.3245,82.76,65,88,5.68,VE,2021-10-01 21:32:13
3,Rikitea,-23.1203,-134.9692,73.17,73,23,22.17,PF,2021-10-01 21:23:36
4,Ilulissat,69.2167,-51.1000,30.22,64,99,0.00,GL,2021-10-01 21:29:10
5,Sabang,5.8933,95.3214,81.28,70,97,8.14,ID,2021-10-01 21:30:52
6,Ewa Beach,21.3156,-158.0072,87.75,50,1,8.05,US,2021-10-01 21:32:13
7,Husavik,66.0449,-17.3389,44.51,98,100,18.52,IS,2021-10-01 21:32:14
8,Tahta,26.7693,31.5021,80.92,36,0,13.82,EG,2021-10-01 21:32:14
9,Pitsunda,43.1592,40.3378,60.98,76,36,13.69,GE,2021-10-01 21:32:15


In [15]:
new_column_order = ["City", "Country", "Date", "Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed"]
city_data_df = city_data_df [new_column_order]
city_data_df.head(10)

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,Bethel,US,2021-10-01 21:30:17,41.3712,-73.4140,66.56,63,20,5.75
1,Altay,CN,2021-10-01 21:24:39,47.8667,88.1167,31.89,69,100,5.14
2,Zaraza,VE,2021-10-01 21:32:13,9.3503,-65.3245,82.76,65,88,5.68
3,Rikitea,PF,2021-10-01 21:23:36,-23.1203,-134.9692,73.17,73,23,22.17
4,Ilulissat,GL,2021-10-01 21:29:10,69.2167,-51.1000,30.22,64,99,0.00
5,Sabang,ID,2021-10-01 21:30:52,5.8933,95.3214,81.28,70,97,8.14
6,Ewa Beach,US,2021-10-01 21:32:13,21.3156,-158.0072,87.75,50,1,8.05
7,Husavik,IS,2021-10-01 21:32:14,66.0449,-17.3389,44.51,98,100,18.52
8,Tahta,EG,2021-10-01 21:32:14,26.7693,31.5021,80.92,36,0,13.82
9,Pitsunda,GE,2021-10-01 21:32:15,43.1592,40.3378,60.98,76,36,13.69


In [16]:
# Create the output file (CSV).
output_data_file = "weather_data/cities.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")